In [2]:
# RNN sweep 
# 2021-02-05

# Settings 

# Import master csv file
input_dir = '~/Downloads/'
csv_name = 'new_RNN_sweep_20210206_10pm.csv'
output_dir = '~/Downloads/'

import pandas as pd
import numpy as np
import os

# Basic data cleaning
rundata = pd.read_csv(os.path.join(input_dir,csv_name))
rundata = rundata[rundata['State']=='finished']


print(len(rundata[rundata['bert_induced']==True]),len(rundata[rundata['bert_induced']==False]))

6360 750


In [3]:
def generate_table(result_file:pd.DataFrame, target, seed_count=False, count_flag=True, std=False, rounding=True):
    
    # Conditions: {5 bert models} x {SingleRNN, CLS_finetune} x {4 items} x {5 targets}

    
    # Possible combinations for our models
    conditionlist = [(result_file['bert_induced'] == False),
                     (result_file['bert_induced'] == True) & (result_file['bert_freeze'] == True) & 
                             (result_file['cls_freeze'] == True),
                     (result_file['bert_induced'] == True) & (result_file['bert_freeze'] == True) & 
                             (result_file['cls_freeze'] == False),
                     (result_file['bert_induced'] == True) & (result_file['bert_freeze'] == False)]
    choicelist = ['singleRNN', 'CLSfixed', 'CLSfinetune','BERTfinetune']
    result_file['model_name'] = np.select(conditionlist, choicelist)
    
    
    
    result_file = result_file.loc[:, ['model_name','bert_model','bert_induced','seed', 'source_file', 'target','item',
                                      'eicu_test_auprc', 'mimic_test_auprc', 'test_auprc']]
    
    #print(result_file.loc['bert_model', result_file['model_name']=='singleRNN'].unique())
    result_filter = result_file.loc[result_file['target']==target]
     
    result_mean = pd.pivot_table(result_filter,
                               columns = ['source_file', 'model_name'],
                               index = ['item', 'bert_model'],
                               values = ['test_auprc', 'mimic_test_auprc', 'eicu_test_auprc'],
                               aggfunc=['mean'])
    
    result_std = pd.pivot_table(result_filter,
                               columns = ['source_file', 'model_name'],
                               index = ['item', 'bert_model'],
                               values = ['test_auprc','mimic_test_auprc', 'eicu_test_auprc'],
                               aggfunc=['std'])
    
    result_count = pd.pivot_table(result_filter,
                               columns = ['source_file', 'model_name'],
                               index = ['item', 'bert_model'],
                               values = ['test_auprc','mimic_test_auprc', 'eicu_test_auprc'],
                               aggfunc=['count'])
    
    if count_flag == True:
        if rounding==True:
            RM_drop = result_mean.droplevel([0], axis=1).round(3)
            RS_drop = result_std.droplevel([0], axis=1).round(3)
        elif rounding == False:
            RM_drop = result_mean.droplevel([0], axis=1)
            RS_drop = result_std.droplevel([0], axis=1)
        RC_drop = result_count.droplevel([0], axis=1)
        
        RC_drop = RC_drop[RM_drop.columns]
        
        for col in range(len(RM_drop.columns)): 
            RM_drop.iloc[:,col][RC_drop.iloc[:,col] > 10] = 'over'
            RM_drop.iloc[:,col][RC_drop.iloc[:,col] < 10] = 'under'   
            
            RS_drop.iloc[:,col][RC_drop.iloc[:,col] > 10] = 'over'
            RS_drop.iloc[:,col][RC_drop.iloc[:,col] < 10] = 'under'  
    
    
    if seed_count==False:
        if count_flag == True:
            if std==True:
                return RS_drop
            elif std==False:
                return RM_drop
        elif count_flag == False:
            if std==True:
                return result_std
            elif std==False:
                return result_mean
    
    elif seed_count==True:
        return RC_drop


In [4]:
target_list = ['readmission', 'mortality', 'los>3day', 'los>7day', 'dx_depth1_unique']

keep_seed = False
count_seeds = False
flag = True

for target in target_list:
    if keep_seed:
        generate_table(rundata, target).iloc[:, ::-1].to_csv(os.path.join(output_dir,'RNNsweep_{}_withseed.csv'.format(target)))
    else:
        if count_seeds==False:
            if flag==True:
                generate_table(rundata, target).iloc[:, ::-1].to_csv(os.path.join(output_dir,'RNNsweep_{}_flagged.csv'.format(target)))
            elif flag==False:
                generate_table(rundata, target).iloc[:, ::-1].to_csv(os.path.join(output_dir,'RNNsweep_{}.csv'.format(target)))
        if count_seeds==True:
            generate_table(rundata, target).iloc[:, ::-1].to_csv(os.path.join(output_dir,'RNNsweep_{}_seedcount.csv'.format(target)))
    

<ipython-input-3-e992721084c9>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RM_drop.iloc[:,col][RC_drop.iloc[:,col] > 10] = 'over'
<ipython-input-3-e992721084c9>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RM_drop.iloc[:,col][RC_drop.iloc[:,col] < 10] = 'under'
<ipython-input-3-e992721084c9>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RS_drop.iloc[:,col][RC_drop.iloc[:,col] > 10] = 'over'
<ipython-input-3-e992721084c9>

In [5]:
generate_table(rundata, 'mortality')

<ipython-input-3-e992721084c9>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RM_drop.iloc[:,col][RC_drop.iloc[:,col] > 10] = 'over'
<ipython-input-3-e992721084c9>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RM_drop.iloc[:,col][RC_drop.iloc[:,col] < 10] = 'under'
<ipython-input-3-e992721084c9>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RS_drop.iloc[:,col][RC_drop.iloc[:,col] > 10] = 'over'
<ipython-input-3-e992721084c9>

eicu_test_auprc                    mimic_test_auprc  \
source_file                          both                                both   
model_name                    CLSfinetune CLSfixed singleRNN      CLSfinetune   
item    bert_model                                                              
all     bert                        0.199    under       NaN            0.428   
        bio_bert                    0.203    under       NaN            0.430   
        bio_clinical_bert           0.205    under       NaN            0.433   
        blue_bert                   0.210    under       NaN            0.423   
        pubmed_bert                 0.201    under     0.188            0.430   
inf     bert                        0.260    under       NaN            0.279   
        bio_bert                    0.256    under       NaN            0.277   
        bio_clinical_bert           0.267    under       NaN            0.279   
        blue_bert                   0.260    under       NaN            0.279   
        pubmed_bert                 0.245    under     0.220            0.280   
lab     bert                        0.203    0.186       NaN            0.282   
        bio_bert                    0.199    0.181       NaN            0.286   
        bio_clinical_bert           0.201    0.194       NaN            0.287   
        blue_bert                   0.203    0.205       NaN            0.289   
        pubmed_bert                 0.195    0.193     0.189            0.286   
lab_inf bio_bert                    0.218      NaN       NaN            0.297   
        bio_clinical_bert           0.211      NaN     0.182            0.299   
        pubmed_bert                 0.213      NaN       NaN            0.302   
med     bert                        0.227    0.165       NaN            0.394   
        bio_bert                    0.210    0.182       NaN            0.395   
        bio_clinical_bert           0.225    0.194       NaN            0.395   
        blue_bert                   0.222    0.204       NaN            0.387   
        pubmed_bert                 0.217    0.201     0.169            0.389   

                                              test_auprc                     \
source_file                                         eicu                      
model_name                CLSfixed singleRNN CLSfinetune CLSfixed singleRNN   
item    bert_model                                                            
all     bert                 under       NaN       0.200    0.139       NaN   
        bio_bert             under       NaN       0.205    0.171       NaN   
        bio_clinical_bert    under       NaN       0.206    0.174       NaN   
        blue_bert            under       NaN       0.204    0.181       NaN   
        pubmed_bert          under     0.390       0.203    0.167     0.187   
inf     bert                 under       NaN       0.265    0.253       NaN   
        bio_bert             under       NaN       0.264    0.263       NaN   
        bio_clinical_bert    under       NaN       0.267    0.275       NaN   
        blue_bert            under       NaN       0.256    0.271       NaN   
        pubmed_bert          under     0.255       0.266    0.256     0.228   
lab     bert                 0.248       NaN       0.222    0.199       NaN   
        bio_bert             0.254       NaN       0.217    0.207       NaN   
        bio_clinical_bert     0.26       NaN       0.226    0.218       NaN   
        blue_bert            0.262       NaN       0.225    0.224       NaN   
        pubmed_bert          0.262     0.296       0.219    0.198     0.183   
lab_inf bio_bert               NaN       NaN       0.231      NaN       NaN   
        bio_clinical_bert      NaN     0.274       0.228      NaN     0.163   
        pubmed_bert            NaN       NaN       0.227      NaN       NaN   
med     bert                 0.345       NaN       0.212    0.173       NaN   
        bio_bert     

In [6]:
generate_table(rundata, 'mortality').iloc[:, ::-1]

<ipython-input-3-e992721084c9>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RM_drop.iloc[:,col][RC_drop.iloc[:,col] > 10] = 'over'
<ipython-input-3-e992721084c9>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RM_drop.iloc[:,col][RC_drop.iloc[:,col] < 10] = 'under'
<ipython-input-3-e992721084c9>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RS_drop.iloc[:,col][RC_drop.iloc[:,col] > 10] = 'over'
<ipython-input-3-e992721084c9>

test_auprc                                          \
source_file                    mimic                           eicu            
model_name                 singleRNN CLSfixed CLSfinetune singleRNN CLSfixed   
item    bert_model                                                             
all     bert                     NaN    0.359       0.439       NaN    0.139   
        bio_bert                 NaN    0.415       0.436       NaN    0.171   
        bio_clinical_bert        NaN    0.419       0.443       NaN    0.174   
        blue_bert                NaN    0.396       0.432       NaN    0.181   
        pubmed_bert            0.402    0.433       0.436     0.187    0.167   
inf     bert                     NaN    0.245       0.277       NaN    0.253   
        bio_bert                 NaN    under       0.278       NaN    0.263   
        bio_clinical_bert        NaN    0.276       0.278       NaN    0.275   
        blue_bert                NaN    0.275       0.279       NaN    0.271   
        pubmed_bert            0.256     0.28       0.282     0.228    0.256   
lab     bert                     NaN    0.264       0.303       NaN    0.199   
        bio_bert                 NaN    0.261       0.302       NaN    0.207   
        bio_clinical_bert        NaN    0.271       0.306       NaN    0.218   
        blue_bert                NaN    0.279       0.313       NaN    0.224   
        pubmed_bert            0.301    0.275       0.309     0.183    0.198   
lab_inf bio_bert                 NaN      NaN       0.312       NaN      NaN   
        bio_clinical_bert      0.286      NaN       0.310     0.163      NaN   
        pubmed_bert              NaN      NaN       0.314       NaN      NaN   
med     bert                     NaN    0.342       0.399       NaN    0.173   
        bio_bert                 NaN     0.37       0.389       NaN    0.198   
        bio_clinical_bert        NaN    0.369       0.394       NaN    0.206   
        blue_bert                NaN    0.379       0.394       NaN    0.206   
        pubmed_bert            0.309    0.396       0.394     0.172    0.205   

                                      mimic_test_auprc                       \
source_file                                       both                        
model_name                CLSfinetune        singleRNN CLSfixed CLSfinetune   
item    bert_model                                                            
all     bert                    0.200              NaN    under       0.428   
        bio_bert                0.205              NaN    under       0.430   
        bio_clinical_bert       0.206              NaN    under       0.433   
        blue_bert               0.204              NaN    under       0.423   
        pubmed_bert             0.203            0.390    under       0.430   
inf     bert                    0.265              NaN    under       0.279   
        bio_bert                0.264              NaN    under       0.277   
        bio_clinical_bert       0.267              NaN    under       0.279   
        blue_bert               0.256              NaN    under       0.279   
        pubmed_bert             0.266            0.255    under       0.280   
lab     bert                    0.222              NaN    0.248       0.282   
        bio_bert                0.217              NaN    0.254       0.286   
        bio_clinical_bert       0.226              NaN     0.26       0.287   
        blue_bert               0.225              NaN    0.262       0.289   
        pubmed_bert             0.219            0.296    0.262       0.286   
lab_inf bio_bert                0.231              NaN      NaN       0.297   
        bio_clinical_bert       0.228            0.274      NaN       0.299   
        pubmed_bert             0.227              NaN      NaN       0.302   
med     bert                    0.212              NaN    0.345       0.394   
        bio_bert                0.201           

In [1]:
#generate_table(rundata, 'mortality').iloc[:, ::-1].to_latex(multirow=True)